In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import altair as alt

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
# Goodbye ryan


In [ ]:
select 
    snippet_title,
    statistics_view_count,
    statistics_like_count,
    statistics_comment_count,
    regexp_substr(player_embed_html, 'youtube.com/[^"]+') as youtube_url
from
    raw.youtube_analytics.video 
where
    ID = 'WEklZ63mMr4'
    or ID ='jK7BRX0j3Qk'
order by
    statistics_view_count asc

In [ ]:
SELECT
  DATE_TRUNC ('MONTH', date) AS month_date,
  MONTHNAME (DATE_TRUNC ('MONTH', date)) AS month_name,
  SUM(total_users) AS total_users
FROM
  RAW.GOOGLE_ANALYTICS.PAGES_PATH_REPORT
GROUP BY
  month_date
ORDER BY
  month_date desc

In [ ]:
SELECT
    REPLACE(page_path, '/tutorials/snowflake/', '') AS page_path,
    SUM(screen_page_views) AS total_screen_page_views
FROM
    RAW.GOOGLE_ANALYTICS.pages_path_report
WHERE
    REPLACE(page_path, '/tutorials/snowflake/', '') != '/tutorials/'
    
GROUP BY
    REPLACE(page_path, '/tutorials/snowflake/', '')
HAVING
    SUM(screen_page_views) > 30
ORDER BY
    total_screen_page_views DESC;

In [ ]:
with clicks as (
    select 
        sum(clicks) as clicks, bitlink_id
    from
        raw.bitly.link_click
    group by
        all
)

select 
    clicks.*, bitlink.title,bitlink.long_url
from
    clicks
left join
    raw.bitly.bitlink as bitlink
        on clicks.bitlink_id = bitlink.id;

# Youtube and Tutorials Analytics

The website "Daniel Wilczak - Tutorials" offers a comprehensive collection of tutorials primarily focused on Snowflake, a cloud data platform. It includes various guides on setting up and using Snowflake with different services such as AWS, Azure, Google Cloud, and APIs. Each tutorial is supplemented with a video for visual learners. Additionally, it covers topics like Git, container services, and local Jupyter Notebooks. The tutorials are user-generated and not officially affiliated with Snowflake.

For more details, you can visit the [website](https://sfc-gh-dwilczak.github.io/tutorials/) here.

In [ ]:
df = cells.youtube_videos.to_pandas()

st.markdown(f"# Youtube Video Metrics")

for index, row in df.iterrows():
    st.markdown(f"### Video: {row['SNIPPET_TITLE']}")

    col1, col2 = st.columns([3, 2])  # Adjust the column width ratio as needed

    with col1:
        st.write(f"**Title:** {row['SNIPPET_TITLE']}")
        st.write(f"**URL:** [Watch Video](https://{row['YOUTUBE_URL']})")

    with col2:
        col1, col2, col3 = st.columns(3)
        with col1:
            st.metric(label="Views", value=row['STATISTICS_VIEW_COUNT'])
        with col2:
            st.metric(label="Likes", value=row['STATISTICS_LIKE_COUNT'])
        with col3:
            st.metric(label="Comments", value=row['STATISTICS_COMMENT_COUNT'])

    st.markdown("---")  # Add a horizontal line to separate videos

In [ ]:
df = cells.bitly.to_pandas()

st.markdown(f"# Bitly links")

for index, row in df.iterrows():
    st.markdown(f"#### [{row['TITLE']}]({row['LONG_URL']}) ")
    st.markdown(f"Clicks:  {row['CLICKS']} ")


In [ ]:
# Graph
df = cells.query_total_users.to_pandas()

df = df.sort_values('MONTH_DATE')

st.subheader('Tutorial Website - Total Users by Month')
total_users_horizontal_bar_chart = alt.Chart(df).mark_bar().encode(
    x='TOTAL_USERS',
    y='MONTH_DATE'
).properties(
    width=800,
    height=200
)

st.altair_chart(total_users_horizontal_bar_chart, use_container_width=True)

In [ ]:
df = cells.query_page_path_views.to_pandas()

## Bar Chart
st.subheader('Tutorial Website - Views By Page')
horizontal_bar_chart = alt.Chart(df).mark_bar().encode(
    x='TOTAL_SCREEN_PAGE_VIEWS',
    y=alt.Y('PAGE_PATH', sort='-x' , axis=alt.Axis(labelLimit=400))
).properties(
    width=800,
    height=500
)

st.altair_chart(horizontal_bar_chart, use_container_width=True)
